# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [39]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json
import geoviews as gv

# Import API key
from api_keys import geoapify_key

In [40]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (°F)
0,0,grytviken,-54.2811,-36.5092,36.032,97,100,5.91,GS,1696263299,36.032
1,1,selfoss,63.9331,-20.9971,50.036,61,86,6.23,IS,1696263246,50.036
2,2,korla,41.7597,86.1469,67.316,30,83,3.01,CN,1696263452,67.316
3,3,tiksi,71.6872,128.8694,26.564,85,100,3.66,RU,1696263146,26.564
4,4,kingston,17.9970,-76.7936,92.246,68,20,1.54,JM,1696263297,92.246


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [51]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="holoviews")

In [52]:
#%%capture --no-display

# Configure the map plot

map_plot = city_data_df.hvplot.points(x='Lng', 
                                         y='Lat', 
                                         c='Humidity', 
                                         cmap='viridis', 
                                         size='Humidity', 
                                         title='City Humidity', 
                                         colorbar=True, 
                                         hover_cols=['City', 'Country', 'Humidity'],
                                         width=800, height=600,
                                         tiles='OSM')

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
max_temp_criteria = (city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] < 80)
humidity_criteria = city_data_df['Humidity'] < 50
cloudiness_criteria = city_data_df['Cloudiness'] < 10
wind_speed_criteria = city_data_df['Wind Speed'] < 10

ideal_cities_df = city_data_df[max_temp_criteria & humidity_criteria & cloudiness_criteria & wind_speed_criteria]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (°F)
35,35,mahalapye,-23.1041,26.8142,73.220,37,0,7.96,BW,1696263462,73.220
59,59,'alaqahdari dishu,30.4374,63.3130,77.576,12,0,1.45,AF,1696263470,77.576
90,90,palapye,-22.5460,27.1251,72.608,40,0,7.84,BW,1696263481,72.608
168,168,springbok,-29.6643,17.8865,79.106,21,0,3.27,ZA,1696263507,79.106
220,220,saranac lake,44.3295,-74.1313,70.124,44,0,2.06,US,1696263517,70.124
342,342,mahriz,31.5917,54.4318,79.304,14,2,1.44,IR,1696263546,79.304
378,378,molepolole,-24.4066,25.4951,79.016,25,0,6.61,BW,1696263555,79.016
473,473,marsabit,2.3284,37.9899,77.810,34,7,8.34,KE,1696263577,77.810
565,565,buckeye,33.3703,-112.5838,78.818,36,2,3.24,US,1696263404,78.818
596,596,karratha,-20.7377,116.8463,79.808,26,2,4.31,AU,1696263246,79.808


In [43]:
# Convert Max Temp from Kelvin to Fahrenheit
#city_data_df['Max Temp (°F)'] = (city_data_df['Max Temp'] - 273.15) * 9/5 + 32
# I did in WeatherPy

# Check the range of converted temperatures
print("Min Temperature (°F):", city_data_df['Max Temp'].min())
print("Max Temperature (°F):", city_data_df['Max Temp'].max())

# Now, you can use the 'Max Temp (°F)' column for your temperature criteria
max_temp_criteria = (city_data_df['Max Temp'] >= 70) & (city_data_df['Max Temp'] <= 80)

# Print the number of cities meeting the temperature criteria
print("Cities with Max Temp between 70°F and 80°F:", sum(max_temp_criteria))

# Other criteria (humidity, cloudiness, wind speed) remain the same
humidity_criteria = city_data_df['Humidity'] < 70
cloudiness_criteria = city_data_df['Cloudiness'] < 10
wind_speed_criteria = city_data_df['Wind Speed'] < 10

# Apply the criteria to filter the DataFrame
ideal_cities_df = city_data_df[max_temp_criteria & humidity_criteria & cloudiness_criteria & wind_speed_criteria]

# Drop any rows with null values (if needed)
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df

Min Temperature (°F): 18.068000000000087
Max Temperature (°F): 108.42800000000008
Cities with Max Temp between 70°F and 80°F: 144


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Max Temp (°F)
35,35,mahalapye,-23.1041,26.8142,73.220,37,0,7.96,BW,1696263462,73.220
59,59,'alaqahdari dishu,30.4374,63.3130,77.576,12,0,1.45,AF,1696263470,77.576
90,90,palapye,-22.5460,27.1251,72.608,40,0,7.84,BW,1696263481,72.608
109,109,bethel,41.3712,-73.4140,76.208,66,0,2.06,US,1696262949,76.208
130,130,tobruk,32.0836,23.9764,76.370,55,0,6.32,LY,1696263260,76.370
168,168,springbok,-29.6643,17.8865,79.106,21,0,3.27,ZA,1696263507,79.106
220,220,saranac lake,44.3295,-74.1313,70.124,44,0,2.06,US,1696263517,70.124
260,260,jamestown,42.0970,-79.2353,77.018,61,0,0.89,US,1696263464,77.018
269,269,rio grande,-32.0350,-52.0986,71.708,67,0,2.73,BR,1696263187,71.708
274,274,sankt veit an der glan,46.7681,14.3603,74.786,64,0,1.03,AT,1696263528,74.786


### Step 3: Create a new DataFrame called `hotel_df`.

In [44]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df.loc[:,['City', 'Country', 'Lat', 'Lng', 'Humidity']]


# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(**{'Hotel Name': ''})

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
35,mahalapye,BW,-23.1041,26.8142,37,
59,'alaqahdari dishu,AF,30.4374,63.3130,12,
90,palapye,BW,-22.5460,27.1251,40,
109,bethel,US,41.3712,-73.4140,66,
130,tobruk,LY,32.0836,23.9764,55,
168,springbok,ZA,-29.6643,17.8865,21,
220,saranac lake,US,44.3295,-74.1313,44,
260,jamestown,US,42.0970,-79.2353,61,
269,rio grande,BR,-32.0350,-52.0986,67,
274,sankt veit an der glan,AT,46.7681,14.3603,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [45]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
mahalapye - nearest hotel: Cresta Mahalapye
'alaqahdari dishu - nearest hotel: No hotel found
palapye - nearest hotel: Tapinda Lodge
bethel - nearest hotel: Hotel Zero Degrees
tobruk - nearest hotel: فندق الاتحاد -طبرق
springbok - nearest hotel: Hotel Springbok Inn
saranac lake - nearest hotel: Hotel Saranac
jamestown - nearest hotel: Chautauqua Harbor Hotel
rio grande - nearest hotel: Hotel Atlântico
sankt veit an der glan - nearest hotel: Glantalerhof
presque isle - nearest hotel: Presque Isle Inn and Convention Center
mastic beach - nearest hotel: SpringHill Suites by Marriott Long Island Brookhaven
tipasa - nearest hotel: Hôtel Matarès
mahriz - nearest hotel: هتل رستوران شاه عباسی
molepolole - nearest hotel: No hotel found
anacapri - nearest hotel: Caesar Augustus
freeport - nearest hotel: Hilton Garden Inn Westbury
marsabit - nearest hotel: Gof Hotel
thief river falls - nearest hotel: GrandStay® Hotel & Suites - Thief River Falls
nouadhibou - nearest hotel: ا

,City,Country,Lat,Lng,Humidity,Hotel Name
35,mahalapye,BW,-23.1041,26.8142,37,Cresta Mahalapye
59,'alaqahdari dishu,AF,30.4374,63.3130,12,No hotel found
90,palapye,BW,-22.5460,27.1251,40,Tapinda Lodge
109,bethel,US,41.3712,-73.4140,66,Hotel Zero Degrees
130,tobruk,LY,32.0836,23.9764,55,فندق الاتحاد -طبرق
168,springbok,ZA,-29.6643,17.8865,21,Hotel Springbok Inn
220,saranac lake,US,44.3295,-74.1313,44,Hotel Saranac
260,jamestown,US,42.0970,-79.2353,61,Chautauqua Harbor Hotel
269,rio grande,BR,-32.0350,-52.0986,67,Hotel Atlântico
274,sankt veit an der glan,AT,46.7681,14.3603,64,Glantalerhof


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [54]:
#%%capture --no-display

# Configure the map plot

# Add 'Hotel Name' and 'Country' to the hover_cols parameter
hotel_plot = hotel_df.hvplot.points(x='Lng', 
                                         y='Lat', 
                                         c='Humidity', 
                                         cmap='viridis', 
                                         size='Humidity', 
                                         colorbar=True, 
                                         hover_cols=['City', 'Country', 'Humidity', 'Hotel Name'],  # Add 'Hotel Name' and 'Country'
                                         width=800, height=600,
                                         tiles='OSM')




# Display the map
hotel_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)